# Photo Conductivity

When the electronic state of the defect interacts with radiation, carriers can be excited to the conduction band. This is called photoconductivity. To help elucidate how likely a defect is to cause photoconductivity, we can look at the band-decomposed contribution to the frequency-dependent dielectric function.
Here we have replicated `VASP`'s density-density optical response calculated in the independent-particle picture in the `pymatgen.io.vasp.optics` module, and a allowed users to control which dipole matrix elements are accounted for by masking the dipole matrix elements stored in the `WAVEDER` file and manipulate the Fermi level to populate or depopulate the defect state.
This essentially allows us to calculate the optical response for (defect)→(conduction band) and (valence band)→(defect) transitions in isolation.

For more information on how `VASP` handles optical response, see the [VASP documentation](https://www.vasp.at/wiki/index.php/LOPTICS).

For more details on how the python code works, please see the documentation for the `pymatgen.io.vasp.optics` module.


In [ ]:
from pathlib import Path
from pymatgen.analysis.defects.ccd import HarmonicDefect, get_SRH_coefficient
from pymatgen.io.vasp.optics import DielectricFunctionCalculator
import numpy as np
from matplotlib import pyplot as plt
TEST_FILES = Path("../../../tests/test_files/v_Ga/")

To compute the optical response of a 

In [ ]:
dir0 = TEST_FILES / "ccd_0_-1" / 1 
hd0 = Har